<div class='alert alert-info'>
<h1>Инструкция</h1>

1) Ваша задача завершить 2 функции: `matrix_multiply` и `matrix_multiply_cuda`.
2) Не изменяйте вообще никакие ячейки, если явно не сказано обратное.
3) Обратите внимание, что в лабе необходима CUDA. Если на вашем устройстве она недоступна, вылетит ошибка. В таком случае, пожалуйста, используйте другое устройство. Также можно использовать Google Colab. Если нет вообще никаких вариантов использовать устройство с CUDA, напишите код и отправьте его без проверок.
4) Для сдачи необходимо отправить письмо на почту spinteh.data.analysis@gmail.com.  
**Тема письма**: ФИО, номер ЛР и группа, например *Иванов Иван Иванович ЛР 2, группа 5*.  
**Контент** прикрепите текстовый файл (только `.py` или `.txt`), который содержит 2 функции (которые вы дописали). Или можно использовать сервисы вставки кода, например, [hastebin](https://www.toptal.com/developers/hastebin).
5) CPU версию (функция `matrix_multiply`) необходимо реализовать самим (через циклы). Использовать готовые функции нельзя.
</div>

In [ ]:
from IPython.display import clear_output

: 

In [ ]:
%pip install -q numpy==1.26.4
%pip install -U -q numba

clear_output()

In [ ]:
import numpy as np
from numba import cuda

In [ ]:
if not cuda.is_available():
    raise ValueError('CUDA is not available. You will not be able to complete the lab.')

In [ ]:
A = np.array([[1, 2, 3],
              [4, 5, 6]], dtype=np.float32)

B = np.array([[7, 8],
              [9, 10],
              [11, 12]], dtype=np.float32)

## CPU

In [ ]:
def matrix_multiply(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    """Function to multiply matrices on CPU

    Args:
        A (np.ndarray): first matrix to multiply
        B (np.ndarray): second matrix to multiply

    Raises:
        ValueError: if the matrices cannot be multiplied due to their shapes

    Returns:
        np.ndarray: The result of matrix multiplication
    """

    # do NOT change code in the block below
    n, m1 = A.shape
    m2, p = B.shape

    if m1 != m2:
        raise ValueError('Matrices cannot be multiplied')

    m = m1
    res = np.zeros((A.shape[0], B.shape[1]), dtype=np.float32)
    # end of block
    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            res[i, j] = sum([A[i][k] * B[v][j] for k in range(A.shape[0]) for v in range(B.shape[1])])
    # Finish code
    return res

In [ ]:
true_c = np.dot(A, B)
true_c

In [ ]:
cpu_c = matrix_multiply(A, B)
cpu_c

In [ ]:
assert np.allclose(true_c, cpu_c, atol=1e-6), 'Function `matrix_multiply` does not work properly.'

## GPU

In [ ]:
C = np.zeros((A.shape[0], B.shape[1]), dtype=np.float32)

In [ ]:
# Определяем размер блока и сетки
TPB = 16  # Threads per block

**Примечание о функции ниже:**  
Результирующая матрица уже подается на вход (`С`). Она сразу правильной формы. Ваша задача заполнить её правильными значениями. Например, если значение, которое должно находиться на позиции (0, 0) получилось у вас в переменной `tmp`, то его можно записать так:

```python
C[0, 0] = tmp
```

Явно возвращать из функции ничего не нужно.

In [ ]:
@cuda.jit
def matrix_multiply_cuda(A: np.ndarray, B: np.ndarray, C: np.ndarray) -> None:
    """Function to multiply matrices on GPU using CUDA

    Args:
        A (np.ndarray): first matrix to multiply
        B (np.ndarray): second matrix to multiply
        C (np.ndarray): the result of matrix multiplication
    """
    
    row, col = cuda.grid(2)

    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.0

        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]

        C[row, col] = tmp
    # finish code

In [ ]:
# Копируем данные в память GPU
A_global_mem = cuda.to_device(A)
B_global_mem = cuda.to_device(B)
C_global_mem = cuda.to_device(C)

In [ ]:
# Определяем размеры сетки и блоков
threadsperblock = (TPB, TPB)

blockspergrid_x = int(np.ceil(A.shape[0] / threadsperblock[0]))
blockspergrid_y = int(np.ceil(B.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

In [ ]:
# Запускаем ядро CUDA
matrix_multiply_cuda[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)

In [ ]:
# Копируем результат обратно в память CPU
C = C_global_mem.copy_to_host()
C

In [ ]:
assert np.allclose(true_c, C, atol=1e-6), 'Function `matrix_multiply_cuda` does not work properly'